<a href="https://colab.research.google.com/github/TayyabKhan54/Machine-learning-project-/blob/main/machinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
#   PURE PCA EVALUATION ON HANDWRITTEN DIGITS DATASET
#   Assignment | All lines are commented for easy understanding
# ============================================================

# --- Import all required libraries ---
import numpy as np                          # NumPy for numerical operations and arrays
import matplotlib.pyplot as plt             # Matplotlib for plotting all graphs
import matplotlib.gridspec as gridspec      # GridSpec for custom subplot layouts
from sklearn.datasets import load_digits    # Built-in digits dataset (8x8 images, 1797 samples)
from sklearn.preprocessing import StandardScaler  # For feature scaling (zero mean, unit variance)
from sklearn.decomposition import PCA       # Principal Component Analysis algorithm
from sklearn.metrics import mean_squared_error     # MSE to measure reconstruction error
import warnings                             # To suppress non-critical warnings
import os                                   # For operating system interactions (e.g., creating directories)
warnings.filterwarnings('ignore')           # Ignore all warnings for clean output

# Ensure the output directory exists
output_dir = '/mnt/user-data/outputs/'
os.makedirs(output_dir, exist_ok=True)

# ============================================================
#   GLOBAL STYLE SETTINGS (applied to all plots)
# ============================================================
plt.rcParams['font.family'] = 'DejaVu Sans'   # Set clean, readable font for all plots
plt.rcParams['font.size'] = 10                # Base font size for labels and text
plt.rcParams['axes.titlesize'] = 13          # Font size for subplot/chart titles
plt.rcParams['axes.labelsize'] = 11          # Font size for axis labels (x and y)
plt.rcParams['figure.facecolor'] = '#0f0f1a' # Dark navy background for all figures
plt.rcParams['axes.facecolor'] = '#1a1a2e'  # Slightly lighter background inside axes
plt.rcParams['axes.edgecolor'] = '#444466'  # Border color around each plot area
plt.rcParams['text.color'] = '#e0e0ff'      # Light purple-white for all text
plt.rcParams['axes.labelcolor'] = '#e0e0ff' # Same color for axis labels
plt.rcParams['xtick.color'] = '#aaaacc'     # Color for x-axis tick marks and labels
plt.rcParams['ytick.color'] = '#aaaacc'     # Color for y-axis tick marks and labels
plt.rcParams['grid.color'] = '#2a2a4a'      # Subtle grid line color
plt.rcParams['grid.linestyle'] = '--'       # Dashed grid lines
plt.rcParams['grid.alpha'] = 0.7            # Slightly transparent grid lines

# Define a reusable color palette for consistent visuals across all plots
COLORS = {
    'accent1': '#7b5ea7',   # Purple — used for primary bars and lines
    'accent2': '#00c9b1',   # Teal — used for secondary/comparison lines
    'accent3': '#ff6b6b',   # Coral red — used for highlights and thresholds
    'accent4': '#ffd166',   # Gold — used for annotations and markers
    'scatter': [            # 10 distinct colors, one per digit class (0-9)
        '#ff6b6b', '#ffd166', '#06d6a0', '#118ab2', '#a8dadc',
        '#e63946', '#f4a261', '#2a9d8f', '#e9c46a', '#c77dff'
    ]
}

print("=" * 60)
print("   PCA EVALUATION — HANDWRITTEN DIGITS DATASET")
print("=" * 60)

# ============================================================
#   PART 1: DATA INSPECTION
# ============================================================
print("\n📦 PART 1: DATA INSPECTION")
print("-" * 40)

# Load the digits dataset from scikit-learn
# This dataset contains 1797 images of handwritten digits (0–9)
# Each image is 8x8 pixels = 64 pixel intensity values as features
digits = load_digits()

# Extract the feature matrix X (shape: 1797 samples × 64 features)
# Each row is one image, each column is one pixel intensity value (0–16)
X = digits.data

# Extract the target vector y (shape: 1797,)
# Each value is the true digit label: 0, 1, 2, ..., 9
y = digits.target

# Extract the raw images in 2D form for visualization (shape: 1797 × 8 × 8)
images = digits.images

# Report dataset dimensions to the user
total_samples = X.shape[0]   # Number of rows = number of digit samples
total_features = X.shape[1]  # Number of columns = number of pixel features (64)
print(f"  ✅ Total Samples  : {total_samples}")
print(f"  ✅ Total Features : {total_features}")
print(f"  ✅ Image Size     : 8 × 8 pixels")
print(f"  ✅ Digit Classes  : {len(np.unique(y))} (digits 0 through 9)")
print(f"  ✅ Feature Range  : min={X.min():.1f}, max={X.max():.1f}")

# ---- Plot: Display 10 sample digit images (one per digit class) ----
fig1, axes = plt.subplots(2, 5, figsize=(12, 5.5))  # 2 rows × 5 columns = 10 subplots
fig1.patch.set_facecolor('#0f0f1a')                  # Set figure background to dark
fig1.suptitle('PART 1 — Sample Digit Images (one per class 0–9)',
              fontsize=14, color='#e0e0ff',          # Title text color
              fontweight='bold', y=1.02)             # Position title slightly above

# Loop over each digit class (0 through 9) and display one example image
for digit_class in range(10):
    # Find the index of the first image that matches this digit class
    first_index = np.where(y == digit_class)[0][0]

    # Calculate subplot row (0 or 1) and column (0–4) position
    row = digit_class // 5   # Integer division: 0-4 → row 0, 5-9 → row 1
    col = digit_class % 5    # Remainder: cycles 0,1,2,3,4 for both rows

    ax = axes[row, col]                          # Select the specific subplot
    ax.imshow(images[first_index],               # Display the 8×8 image array
              cmap='plasma',                     # Colormap: purple-to-yellow gradient
              interpolation='nearest')           # No blurring between pixels
    ax.set_title(f'Digit: {digit_class}',        # Label the subplot with the digit
                 color='#ffd166',                # Gold color for digit titles
                 fontsize=11, fontweight='bold')
    ax.axis('off')                               # Remove x/y axis ticks and labels

plt.tight_layout()              # Prevent overlapping between subplots
plt.savefig('/mnt/user-data/outputs/part1_sample_images.png',
            dpi=150, bbox_inches='tight',        # Crop whitespace around figure
            facecolor='#0f0f1a')                 # Preserve dark background in saved file
plt.close()                                      # Close figure to free memory
print("\n  💾 Saved: part1_sample_images.png")

# ============================================================
#   PART 2: PCA ANALYSIS
# ============================================================
print("\n📊 PART 2: PCA ANALYSIS")
print("-" * 40)

# ---- Step 2A: Feature Scaling ----
# StandardScaler transforms data so each feature has:
#   mean = 0 (centered) and standard deviation = 1 (unit variance)
# This is critical before PCA because PCA is sensitive to feature scales.
# Without scaling, high-intensity pixels would dominate the principal components.
scaler = StandardScaler()          # Create the scaler object

# fit_transform computes mean & std from X, then transforms X in one step
# The result X_scaled has the same shape as X (1797 × 64) but normalized values
X_scaled = scaler.fit_transform(X)

print("  ✅ Feature scaling applied (StandardScaler: mean=0, std=1)")
print(f"     After scaling — mean ≈ {X_scaled.mean():.6f}, std ≈ {X_scaled.std():.4f}")

# ---- Step 2B: Apply PCA with all 64 components ----
# We first run PCA with ALL components to analyze the full variance structure.
# n_components=64 means we keep all principal components (same as number of features).
# random_state=42 ensures reproducibility of results.
pca_full = PCA(n_components=64, random_state=42)

# fit_transform projects the scaled data into PCA space (still 64 dimensions here)
X_pca_full = pca_full.fit_transform(X_scaled)

# ---- Step 2C: Extract Explained Variance Ratio ----
# explained_variance_ratio_ is an array of 64 values.
# Each value tells us what fraction of total variance is captured by that component.
# Example: [0.12, 0.097, 0.085, ...] means PC1 explains 12%, PC2 explains 9.7%, etc.
evr = pca_full.explained_variance_ratio_        # Array of 64 variance ratios

# Cumulative sum: adds up variance ratios progressively
# cumvar[i] = total variance explained by the first (i+1) principal components
cumvar = np.cumsum(evr)                          # Array of 64 cumulative variance values

print("\n  Top 10 components and their explained variance:")
for i in range(10):       # Print information for the first 10 principal components
    print(f"    PC{i+1:2d}: {evr[i]*100:.2f}% variance | Cumulative: {cumvar[i]*100:.2f}%")

# ---- Step 2D: Determine Minimum Components for 90% and 95% Variance ----
# np.argmax returns the index of the FIRST position where condition is True.
# cumvar >= 0.90 is a boolean array; argmax finds first True index.
# We add 1 because components are 1-indexed (PC1, PC2, ...).
n_components_90 = np.argmax(cumvar >= 0.90) + 1  # Min components for 90% variance
n_components_95 = np.argmax(cumvar >= 0.95) + 1  # Min components for 95% variance

print(f"\n  ✅ Components for 90% variance: {n_components_90}")
print(f"  ✅ Components for 95% variance: {n_components_95}")
print(f"     Actual cumulative variance at {n_components_90} components: {cumvar[n_components_90-1]*100:.2f}%")
print(f"     Actual cumulative variance at {n_components_95} components: {cumvar[n_components_95-1]*100:.2f}%")

# ---- Plot: Scree Plot + Cumulative Variance Plot (side by side) ----
fig2, (ax_scree, ax_cum) = plt.subplots(1, 2, figsize=(14, 5.5))  # Two plots side by side
fig2.patch.set_facecolor('#0f0f1a')  # Dark figure background
fig2.suptitle('PART 2 — PCA Variance Analysis',
              fontsize=14, color='#e0e0ff', fontweight='bold', y=1.02)

# --- Left plot: Scree Plot ---
# A scree plot shows individual explained variance for each component.
# The "elbow" point indicates where adding more components gives diminishing returns.
component_numbers = np.arange(1, 65)       # Component labels: 1, 2, 3, ..., 64

ax_scree.bar(component_numbers,            # x-axis: component index
             evr * 100,                    # y-axis: individual variance % for each
             color=COLORS['accent1'],      # Purple bars
             alpha=0.85,                   # Slightly transparent
             edgecolor='#5a3e8a',          # Darker purple edges on bars
             linewidth=0.5)                # Thin edges

# Add a smooth line on top of the bar chart to show the trend
ax_scree.plot(component_numbers, evr * 100,
              color=COLORS['accent2'],     # Teal line for visual contrast
              linewidth=1.5,              # Moderate line thickness
              marker='o',                 # Circular markers at each data point
              markersize=2,               # Small markers to avoid clutter
              alpha=0.8)                  # Slight transparency

ax_scree.set_title('Scree Plot', color='#e0e0ff', pad=10)
ax_scree.set_xlabel('Principal Component Number')
ax_scree.set_ylabel('Explained Variance (%)')
ax_scree.grid(True, alpha=0.4)            # Show subtle grid lines

# Mark the elbow region (around component 10) with a vertical dashed line
ax_scree.axvline(x=10, color=COLORS['accent4'], linestyle='--',
                 linewidth=1.2, alpha=0.8, label='Elbow ≈ PC10')
ax_scree.legend(fontsize=9, facecolor='#1a1a2e', labelcolor='#e0e0ff')

# Annotate the first few high-variance components with their values
for i in [0, 1, 2]:     # Annotate PC1, PC2, PC3 with their variance %
    ax_scree.annotate(f'{evr[i]*100:.1f}%',
                      xy=(i+1, evr[i]*100),       # Arrow points to the bar top
                      xytext=(i+3, evr[i]*100+0.5),  # Text slightly offset
                      fontsize=8, color=COLORS['accent4'],
                      arrowprops=dict(arrowstyle='->', color=COLORS['accent4'], lw=1.2))

# --- Right plot: Cumulative Explained Variance ---
# This plot shows how total captured variance increases as we add more components.
# Helps us decide the minimum number of components needed to retain a target %.
ax_cum.plot(component_numbers, cumvar * 100,
            color=COLORS['accent2'],       # Teal line for cumulative variance
            linewidth=2.5,                # Thicker line for emphasis
            marker='o', markersize=2)

# Shade the area under the cumulative variance curve for visual impact
ax_cum.fill_between(component_numbers, cumvar * 100,
                    alpha=0.15,            # Very transparent fill
                    color=COLORS['accent2'])

# Draw a horizontal dashed line at the 90% variance threshold
ax_cum.axhline(y=90, color=COLORS['accent3'], linestyle='--',
               linewidth=1.5, label=f'90% ({n_components_90} components)')

# Draw a horizontal dashed line at the 95% variance threshold
ax_cum.axhline(y=95, color=COLORS['accent4'], linestyle='--',
               linewidth=1.5, label=f'95% ({n_components_95} components)')

# Draw a vertical dashed line at the 90% component threshold
ax_cum.axvline(x=n_components_90, color=COLORS['accent3'],
               linestyle=':', linewidth=1.2, alpha=0.8)

# Draw a vertical dashed line at the 95% component threshold
ax_cum.axvline(x=n_components_95, color=COLORS['accent4'],
               linestyle=':', linewidth=1.2, alpha=0.8)

# Add text annotations showing exactly where 90% and 95% are reached
ax_cum.annotate(f'PC{n_components_90}→90%',
                xy=(n_components_90, 90),
                xytext=(n_components_90 + 5, 80),
                color=COLORS['accent3'], fontsize=9,
                arrowprops=dict(arrowstyle='->', color=COLORS['accent3'], lw=1.2))

ax_cum.annotate(f'PC{n_components_95}→95%',
                xy=(n_components_95, 95),
                xytext=(n_components_95 + 5, 85),
                color=COLORS['accent4'], fontsize=9,
                arrowprops=dict(arrowstyle='->', color=COLORS['accent4'], lw=1.2))

ax_cum.set_title('Cumulative Explained Variance', color='#e0e0ff', pad=10)
ax_cum.set_xlabel('Number of Principal Components')
ax_cum.set_ylabel('Cumulative Explained Variance (%)')
ax_cum.set_ylim([0, 102])            # Set y-axis from 0% to slightly above 100%
ax_cum.grid(True, alpha=0.4)
ax_cum.legend(fontsize=9, facecolor='#1a1a2e', labelcolor='#e0e0ff')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/part2_pca_variance.png',
            dpi=150, bbox_inches='tight', facecolor='#0f0f1a')
plt.close()
print("\n  💾 Saved: part2_pca_variance.png")

# ============================================================
#   PART 3: DIMENSIONALITY REDUCTION
# ============================================================
print("\n📉 PART 3: DIMENSIONALITY REDUCTION")
print("-" * 40)

# We now apply PCA with ONLY the number of components needed for 95% variance.
# This reduces the 64-dimensional data to n_components_95 dimensions.
# We use 95% as the primary reduction target for highest quality reconstruction.
pca_95 = PCA(n_components=n_components_95, random_state=42)

# Transform the scaled data into the reduced PCA space
# X_reduced_95 has shape (1797, n_components_95) — much smaller than (1797, 64)
X_reduced_95 = pca_95.fit_transform(X_scaled)

# Also apply PCA for the 90% variance case (for comparison)
pca_90 = PCA(n_components=n_components_90, random_state=42)
X_reduced_90 = pca_90.fit_transform(X_scaled)  # Shape: (1797, n_components_90)

# Compute how much we reduced the dimensionality compared to original 64 features
original_features = X.shape[1]      # Original feature count = 64

# Reduction % = ((original - reduced) / original) × 100
reduction_pct_95 = (1 - n_components_95 / original_features) * 100  # For 95% variance case
reduction_pct_90 = (1 - n_components_90 / original_features) * 100  # For 90% variance case

print(f"  Original Number of Features  : {original_features}")
print(f"\n  [90% Variance Case]")
print(f"  Reduced Number of Features   : {n_components_90}")
print(f"  Dimensionality Reduction     : {reduction_pct_90:.1f}%")
print(f"  Formula: (1 - {n_components_90}/{original_features}) × 100 = {reduction_pct_90:.1f}%")

print(f"\n  [95% Variance Case]")
print(f"  Reduced Number of Features   : {n_components_95}")
print(f"  Dimensionality Reduction     : {reduction_pct_95:.1f}%")
print(f"  Formula: (1 - {n_components_95}/{original_features}) × 100 = {reduction_pct_95:.1f}%")

# ============================================================
#   PART 4: RECONSTRUCTION ERROR
# ============================================================
print("\n🔄 PART 4: RECONSTRUCTION ERROR")
print("-" * 40)

# PCA Reconstruction: We can convert back from reduced space to original 64D space.
# inverse_transform reverses the PCA projection using the stored components.
# The reconstruction is an APPROXIMATION — some information is lost (the discarded variance).
X_reconstructed_95 = pca_95.inverse_transform(X_reduced_95)  # 95% case: shape (1797, 64)
X_reconstructed_90 = pca_90.inverse_transform(X_reduced_90)  # 90% case: shape (1797, 64)

# The reconstructions are in scaled space; we un-scale them back to pixel intensities
# inverse_transform on the scaler reverses the normalization (re-adds mean, re-scales by std)
X_reconstructed_95_orig = scaler.inverse_transform(X_reconstructed_95)
X_reconstructed_90_orig = scaler.inverse_transform(X_reconstructed_90)

# Compute Mean Squared Error (MSE) between original and reconstructed data
# MSE = average of squared differences pixel-by-pixel across all images
mse_95 = mean_squared_error(X, X_reconstructed_95_orig)  # MSE for 95% case
mse_90 = mean_squared_error(X, X_reconstructed_90_orig)  # MSE for 90% case

print(f"  [90% Variance — {n_components_90} components]")
print(f"  Reconstruction MSE  : {mse_90:.4f}")
print(f"  Meaning: Each pixel is off by ≈ {np.sqrt(mse_90):.2f} intensity units on average")

print(f"\n  [95% Variance — {n_components_95} components]")
print(f"  Reconstruction MSE  : {mse_95:.4f}")
print(f"  Meaning: Each pixel is off by ≈ {np.sqrt(mse_95):.2f} intensity units on average")

print(f"\n  ✅ Using more components (95%) reduces MSE by "
      f"{( (mse_90 - mse_95)/mse_90)*100:.1f}% compared to 90% case")

# ---- Plot: Original vs Reconstructed Digit Images Comparison ----
fig4 = plt.figure(figsize=(14, 7))    # Create wide figure for comparison
fig4.patch.set_facecolor('#0f0f1a')
fig4.suptitle('PART 4 — Original vs Reconstructed Digit Images',
              fontsize=14, color='#e0e0ff', fontweight='bold', y=1.01)

# Select 6 diverse digit indices to compare: one from each class 0–5
sample_indices = [np.where(y == d)[0][0] for d in range(6)]  # First sample of digit 0–5

n_samples_show = len(sample_indices)   # Number of images to compare = 6

# We'll show 3 rows: Original | 90% Recon | 95% Recon
# And n_samples_show columns (one per digit)
row_labels = ['Original', f'90% Recon\n({n_components_90} PCs)', f'95% Recon\n({n_components_95} PCs)']
row_data = [X, X_reconstructed_90_orig, X_reconstructed_95_orig]  # Data for each row

# Define border colors for each row to visually distinguish them
row_border_colors = ['#7b5ea7', '#ff6b6b', '#00c9b1']   # Purple, Red, Teal

for row_idx, (data, label, border_col) in enumerate(zip(row_data, row_labels, row_border_colors)):
    for col_idx, sample_idx in enumerate(sample_indices):
        # Calculate the position in a (3 rows × 6 cols) grid of subplots
        # Subplot numbering: (row_idx * n_samples_show + col_idx + 1)
        ax = fig4.add_subplot(3, n_samples_show, row_idx * n_samples_show + col_idx + 1)

        # Reshape the flat 64-element vector back to 8×8 image for display
        img = data[sample_idx].reshape(8, 8)

        ax.imshow(img,              # Show the image
                  cmap='plasma',    # Same colormap as Part 1 for consistency
                  interpolation='nearest',
                  vmin=0, vmax=16)  # Fix color scale to digit dataset range (0–16)

        # Add row label only on the leftmost column
        if col_idx == 0:
            ax.set_ylabel(label, fontsize=9, color=border_col, fontweight='bold')

        # Add digit number as title on the top row only
        if row_idx == 0:
            ax.set_title(f'Digit {y[sample_idx]}',
                         fontsize=10, color='#ffd166', fontweight='bold')

        ax.axis('off')  # Remove axis ticks and labels for cleaner look

        # Draw a colored border around each subplot to match the row's theme
        for spine in ax.spines.values():
            spine.set_edgecolor(border_col)
            spine.set_linewidth(2)
            spine.set_visible(True)

# Add MSE annotations as text below the comparison figure
fig4.text(0.5, -0.03,
          f'MSE (90%): {mse_90:.4f}    |    MSE (95%): {mse_95:.4f}    '
          f'|    Lower MSE = Better Reconstruction',
          ha='center', va='center', fontsize=10,
          color='#e0e0ff', style='italic')

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/part4_reconstruction.png',
            dpi=150, bbox_inches='tight', facecolor='#0f0f1a')
plt.close()
print("\n  💾 Saved: part4_reconstruction.png")

# ============================================================
#   PART 5: 2D VISUALIZATION (PC1 vs PC2)
# ============================================================
print("\n🎨 PART 5: 2D PCA VISUALIZATION")
print("-" * 40)

# For 2D visualization, apply PCA keeping only the first 2 principal components.
# These 2 components capture the most variance and can be plotted on a 2D scatter plot.
pca_2d = PCA(n_components=2, random_state=42)

# Project all 1797 samples into 2D space
# X_2d has shape (1797, 2) — each row is (PC1_value, PC2_value) for one image
X_2d = pca_2d.fit_transform(X_scaled)

# How much total variance do PC1 and PC2 together capture?
var_pc1 = pca_2d.explained_variance_ratio_[0] * 100   # PC1 variance %
var_pc2 = pca_2d.explained_variance_ratio_[1] * 100   # PC2 variance %
total_2d_var = var_pc1 + var_pc2                       # Combined variance %

print(f"  PC1 explains: {var_pc1:.1f}% variance")
print(f"  PC2 explains: {var_pc2:.1f}% variance")
print(f"  Together (2D projection): {total_2d_var:.1f}% of total variance")

# ---- Plot: 2D Scatter Plot (PC1 vs PC2), colored by digit class ----
fig5, ax = plt.subplots(figsize=(11, 8))    # Single large scatter plot
fig5.patch.set_facecolor('#0f0f1a')

# Plot each digit class as a separate scatter series for distinct colors and legend
for digit_class in range(10):                          # Loop over all 10 digit classes
    # Get boolean mask: True where y matches this digit class
    mask = y == digit_class

    ax.scatter(
        X_2d[mask, 0],                 # PC1 values for this digit class (x-axis)
        X_2d[mask, 1],                 # PC2 values for this digit class (y-axis)
        c=COLORS['scatter'][digit_class],  # Unique color for each digit
        label=f'Digit {digit_class}',  # Legend entry for this class
        alpha=0.65,                    # Semi-transparent so overlapping points are visible
        s=22,                          # Marker size (in points²)
        edgecolors='none'              # No border on markers for cleaner look
    )

# Add axis labels with variance percentage to show what each axis represents
ax.set_xlabel(f'Principal Component 1  ({var_pc1:.1f}% variance)', fontsize=12)
ax.set_ylabel(f'Principal Component 2  ({var_pc2:.1f}% variance)', fontsize=12)
ax.set_title(f'2D PCA Projection of Handwritten Digits\n'
             f'(PC1+PC2 capture {total_2d_var:.1f}% of total variance)',
             color='#e0e0ff', fontsize=13, fontweight='bold', pad=14)

# Add a legend explaining which color corresponds to which digit
legend = ax.legend(title='Digit Class',
                   bbox_to_anchor=(1.02, 1),  # Place legend outside plot on the right
                   loc='upper left',
                   fontsize=9,
                   title_fontsize=10,
                   facecolor='#1a1a2e',       # Dark legend background
                   edgecolor='#444466',       # Border of the legend box
                   labelcolor='#e0e0ff')      # Text color in legend
legend.get_title().set_color('#e0e0ff')       # Legend title color

ax.grid(True, alpha=0.3)  # Add subtle grid for easier coordinate reading

# Add an annotation box summarizing key stats on the plot
info_text = (f"Samples: {total_samples}\n"
             f"Features: {original_features} → 2\n"
             f"Variance: {total_2d_var:.1f}%")
ax.text(0.02, 0.97, info_text,
        transform=ax.transAxes,     # Coordinates relative to axes (0-1 range)
        fontsize=9, color='#e0e0ff',
        verticalalignment='top',
        bbox=dict(boxstyle='round,pad=0.5',   # Rounded rectangle box
                  facecolor='#2a2a4a',
                  edgecolor='#7b5ea7',
                  alpha=0.8))

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/part5_2d_visualization.png',
            dpi=150, bbox_inches='tight', facecolor='#0f0f1a')
plt.close()
print("  💾 Saved: part5_2d_visualization.png")

# ============================================================
#   COMBINED SUMMARY DASHBOARD (all results at a glance)
# ============================================================
print("\n📋 Generating Summary Dashboard...")

fig_sum = plt.figure(figsize=(16, 9))     # Large dashboard figure
fig_sum.patch.set_facecolor('#0f0f1a')
fig_sum.suptitle('PCA EVALUATION — HANDWRITTEN DIGITS | SUMMARY DASHBOARD',
                 fontsize=15, color='#e0e0ff', fontweight='bold', y=1.0)

# Use GridSpec for flexible subplot sizing
gs = gridspec.GridSpec(2, 3, figure=fig_sum,  # 2 rows, 3 columns
                       hspace=0.45,           # Vertical spacing between rows
                       wspace=0.35)           # Horizontal spacing between columns

# ---- Panel 1 (top-left): Scree Plot ----
ax1 = fig_sum.add_subplot(gs[0, 0])
ax1.bar(component_numbers[:20], evr[:20]*100, color=COLORS['accent1'], alpha=0.85)
ax1.plot(component_numbers[:20], evr[:20]*100, color=COLORS['accent2'], lw=1.5, marker='o', ms=3)
ax1.set_title('Scree Plot (Top 20 PCs)', pad=8)
ax1.set_xlabel('Component')
ax1.set_ylabel('Variance (%)')
ax1.grid(True, alpha=0.4)

# ---- Panel 2 (top-center): Cumulative Variance ----
ax2 = fig_sum.add_subplot(gs[0, 1])
ax2.plot(component_numbers, cumvar*100, color=COLORS['accent2'], lw=2)
ax2.fill_between(component_numbers, cumvar*100, alpha=0.12, color=COLORS['accent2'])
ax2.axhline(90, color=COLORS['accent3'], ls='--', lw=1.5, label=f'90% → {n_components_90} PCs')
ax2.axhline(95, color=COLORS['accent4'], ls='--', lw=1.5, label=f'95% → {n_components_95} PCs')
ax2.set_title('Cumulative Variance', pad=8)
ax2.set_xlabel('Components')
ax2.set_ylabel('Cumulative (%)')
ax2.set_ylim([0, 102])
ax2.legend(fontsize=8, facecolor='#1a1a2e', labelcolor='#e0e0ff')
ax2.grid(True, alpha=0.4)

# ---- Panel 3 (top-right): Dimensionality Reduction Bar Chart ----
ax3 = fig_sum.add_subplot(gs[0, 2])
cases = ['Original\n(64 dims)', f'90% Var\n({n_components_90} dims)', f'95% Var\n({n_components_95} dims)']
dims = [64, n_components_90, n_components_95]
bar_colors = [COLORS['accent1'], COLORS['accent3'], COLORS['accent4']]
bars = ax3.bar(cases, dims, color=bar_colors, alpha=0.85, width=0.5, edgecolor='#2a2a3a')
# Add value labels on top of each bar
for bar, dim in zip(bars, dims):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             str(dim), ha='center', va='bottom', fontsize=10, color='#e0e0ff', fontweight='bold')
ax3.set_title('Dimensionality Comparison', pad=8)
ax3.set_ylabel('Number of Features')
ax3.grid(True, alpha=0.3, axis='y')

# ---- Panel 4 (bottom-left): 2D Scatter Plot ----
ax4 = fig_sum.add_subplot(gs[1, 0])
for digit_class in range(10):
    mask = y == digit_class
    ax4.scatter(X_2d[mask, 0], X_2d[mask, 1],
                c=COLORS['scatter'][digit_class],
                label=f'{digit_class}', alpha=0.5, s=8, edgecolors='none')
ax4.set_title(f'2D PCA Scatter ({total_2d_var:.1f}% var)', pad=8)
ax4.set_xlabel(f'PC1 ({var_pc1:.1f}%)')
ax4.set_ylabel(f'PC2 ({var_pc2:.1f}%)')
ax4.legend(title='Digit', fontsize=6, title_fontsize=7,
           facecolor='#1a1a2e', labelcolor='#e0e0ff',
           ncol=5, loc='upper right', markerscale=2)
ax4.grid(True, alpha=0.3)

# ---- Panel 5 (bottom-center): Reconstruction Error Bar Chart ----
ax5 = fig_sum.add_subplot(gs[1, 1])
cases_mse = [f'90% Var\n({n_components_90} PCs)', f'95% Var\n({n_components_95} PCs)']
mse_values = [mse_90, mse_95]
mse_bar_colors = [COLORS['accent3'], COLORS['accent4']]
bars5 = ax5.bar(cases_mse, mse_values, color=mse_bar_colors, alpha=0.85,
                width=0.4, edgecolor='#2a2a3a')
for bar, val in zip(bars5, mse_values):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{val:.4f}', ha='center', va='bottom', fontsize=10,
             color='#e0e0ff', fontweight='bold')
ax5.set_title('Reconstruction Error (MSE)', pad=8)
ax5.set_ylabel('Mean Squared Error')
ax5.grid(True, alpha=0.3, axis='y')

# ---- Panel 6 (bottom-right): Key Results Text Summary ----
ax6 = fig_sum.add_subplot(gs[1, 2])
ax6.axis('off')   # This panel is purely text — no axes needed

# Build the summary text block with all key results
summary = (
    "KEY RESULTS\n"
    "─────────────────────────\n"
    f"Dataset: 1797 samples\n"
    f"Original features: 64\n\n"
    f"90% Variance:\n"
    f"  Components: {n_components_90}\n"
    f"  Reduced by: {reduction_pct_90:.1f}%\n"
    f"  MSE: {mse_90:.4f}\n\n"
    f"95% Variance:\n"
    f"  Components: {n_components_95}\n"
    f"  Reduced by: {reduction_pct_95:.1f}%\n"
    f"  MSE: {mse_95:.4f}\n\n"
    f"2D Projection:\n"
    f"  PC1: {var_pc1:.1f}%\n"
    f"  PC2: {var_pc2:.1f}%\n"
    f"  Total: {total_2d_var:.1f}%"
)

# Place the text block inside the empty subplot with a styled box
ax6.text(0.05, 0.95, summary,
         transform=ax6.transAxes,
         fontsize=9.5, color='#e0e0ff',
         verticalalignment='top',
         fontfamily='monospace',           # Monospace for aligned text columns
         bbox=dict(boxstyle='round,pad=0.8',
                   facecolor='#1a1a2e',
                   edgecolor='#7b5ea7',
                   alpha=0.9))

plt.savefig('/mnt/user-data/outputs/summary_dashboard.png',
            dpi=150, bbox_inches='tight', facecolor='#0f0f1a')
plt.close()
print("  💾 Saved: summary_dashboard.png")

# ============================================================
#   FINAL CONSOLE SUMMARY
# ============================================================
print("\n" + "=" * 60)
print("   FINAL RESULTS SUMMARY")
print("=" * 60)
print(f"\n  Dataset : Handwritten Digits (sklearn)")
print(f"  Samples : {total_samples}     Features: {original_features}")
print(f"\n  ┌─────────────────────────────────────────┐")
print(f"  │  Variance  │ Components │  Reduction  │  MSE   │")
print(f"  ├─────────────────────────────────────────┤")
print(f"  │    90%     │     {n_components_90:2d}     │   {reduction_pct_90:5.1f}%   │ {mse_90:.4f} │")
print(f"  │    95%     │     {n_components_95:2d}     │   {reduction_pct_95:5.1f}%   │ {mse_95:.4f} │")
print(f"  └─────────────────────────────────────────┘")
print(f"\n  2D Projection Variance: {total_2d_var:.1f}%")
print(f"  (PC1={var_pc1:.1f}%, PC2={var_pc2:.1f}%)")

print("\n" + "=" * 60)
print("   CONCLUSION")
print("=" * 60)
conclusion = """
This experiment demonstrates the effectiveness of PCA for
dimensionality reduction on the handwritten digits dataset.

The original 64-pixel features can be reduced to just
{n90} components while retaining 90% of variance, and
{n95} components for 95% — reductions of {r90:.1f}% and
{r95:.1f}% respectively. This dramatically reduces storage
and computation while preserving most information.

Reconstruction error (MSE) confirms this tradeoff: the
90% case has higher error ({m90:.4f}) while the 95% case
achieves lower error ({m95:.4f}) with only a few more
components. Visual inspection shows 95% reconstructions
are nearly indistinguishable from originals.

The 2D scatter plot reveals meaningful clustering — digit
classes form loose but distinct regions in PC1-PC2 space,
confirming that PCA captures true structure in the data
even without using class labels (unsupervised). Digits
like 0 and 1 are more isolated, while 3, 5, 8 show more
overlap due to similar stroke patterns.

Overall, PCA proves to be a powerful, interpretable tool
for compressing high-dimensional image data with minimal
information loss and clear geometric insight.
""".format(n90=n_components_90, n95=n_components_95,
           r90=reduction_pct_90, r95=reduction_pct_95,
           m90=mse_90, m95=mse_95)
print(conclusion)
print("  ALL PLOTS SAVED SUCCESSFULLY ✅")
print("=" * 60)


   PCA EVALUATION — HANDWRITTEN DIGITS DATASET

📦 PART 1: DATA INSPECTION
----------------------------------------
  ✅ Total Samples  : 1797
  ✅ Total Features : 64
  ✅ Image Size     : 8 × 8 pixels
  ✅ Digit Classes  : 10 (digits 0 through 9)
  ✅ Feature Range  : min=0.0, max=16.0

  💾 Saved: part1_sample_images.png

📊 PART 2: PCA ANALYSIS
----------------------------------------
  ✅ Feature scaling applied (StandardScaler: mean=0, std=1)
     After scaling — mean ≈ 0.000000, std ≈ 0.9763

  Top 10 components and their explained variance:
    PC 1: 12.03% variance | Cumulative: 12.03%
    PC 2: 9.56% variance | Cumulative: 21.59%
    PC 3: 8.44% variance | Cumulative: 30.04%
    PC 4: 6.50% variance | Cumulative: 36.54%
    PC 5: 4.86% variance | Cumulative: 41.40%
    PC 6: 4.21% variance | Cumulative: 45.61%
    PC 7: 3.94% variance | Cumulative: 49.55%
    PC 8: 3.39% variance | Cumulative: 52.94%
    PC 9: 3.00% variance | Cumulative: 55.94%
    PC10: 2.93% variance | Cumulative: 